In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 10, 12)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-10-12
yesterday: 2023-10-11


In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1338945,WHART,2023-10-11,2023-10-11,0,9.65,9.70,0.05,0.52%,1,0,9.90,9.60,"6,443,146",2023-10-11,2023-10-11,1


In [5]:
names = tmp["name"]
name = names.to_string(index=False)
name

'WHART'

In [6]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,128,WHART,0.00,0.00,B,8.80,12.20,0.00,0.00,12.30,"5,000",-4,4,"20,000",0.000000,0.000000,RD30pct,SET


### End of Tables in the process

In [7]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-10-11'
ORDER BY a.name



(48, 1)

In [8]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-10-11' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [9]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,ASK,2023-10-10,2023-10-11,20.30,21.30,2903164,22.00,20.30,B,SET,1.00,4.93
1,DIF,2023-10-11,2023-10-11,9.05,9.10,46019438,9.10,9.00,B,SET,0.05,0.55
2,JASIF,2023-10-11,2023-10-11,6.65,6.80,17166120,6.80,6.65,B,SET,0.15,2.26
3,JMT,2023-10-06,2023-10-11,46.25,40.75,269784432,50.00,40.75,B,SET50,-5.50,-11.89
4,NER,2023-10-10,2023-10-11,4.46,4.56,20767712,4.56,4.46,B,SET,0.10,2.24
5,ORI,2023-10-10,2023-10-11,9.75,10.20,13089996,10.20,9.75,B,SET100,0.45,4.62
6,PTG,2023-10-11,2023-10-11,8.80,9.00,36446964,9.20,8.80,B,SET100,0.20,2.27
7,PTTGC,2023-10-10,2023-10-11,33.00,34.25,59101876,34.25,31.75,B,SET50,1.25,3.79
8,SENA,2023-10-11,2023-10-11,2.64,2.66,25069508,2.80,2.64,B,SET,0.02,0.76
9,SINGER,2023-10-11,2023-10-11,11.30,11.60,158941944,13.40,11.30,B,SET100,0.30,2.65


### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [10]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,ASK,2023-10-10,2023-10-11,20.30,21.30,"2,903,164",22.00,20.30,4.93%,B
1,DIF,2023-10-11,2023-10-11,9.05,9.10,"46,019,438",9.10,9.00,0.55%,B
2,JASIF,2023-10-11,2023-10-11,6.65,6.80,"17,166,120",6.80,6.65,2.26%,B
3,JMT,2023-10-06,2023-10-11,46.25,40.75,"269,784,432",50.00,40.75,-11.89%,B
4,NER,2023-10-10,2023-10-11,4.46,4.56,"20,767,712",4.56,4.46,2.24%,B
5,ORI,2023-10-10,2023-10-11,9.75,10.20,"13,089,996",10.20,9.75,4.62%,B
6,PTG,2023-10-11,2023-10-11,8.80,9.00,"36,446,964",9.20,8.80,2.27%,B
7,PTTGC,2023-10-10,2023-10-11,33.00,34.25,"59,101,876",34.25,31.75,3.79%,B
8,SENA,2023-10-11,2023-10-11,2.64,2.66,"25,069,508",2.80,2.64,0.76%,B
9,SINGER,2023-10-11,2023-10-11,11.30,11.60,"158,941,944",13.40,11.30,2.65%,B


In [11]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [12]:
sales = df[cols]
sales.shape

(48, 10)

In [13]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [14]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(48, 19)

In [15]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,ASK,2023-10-10,2023-10-11,20.30,21.30,2903164,22.00,20.30,4.93,B,21.20,-0.10,410235,2023-10-12
1,DIF,2023-10-11,2023-10-11,9.05,9.10,46019438,9.10,9.00,0.55,B,9.05,-0.05,11982216,2023-10-12
2,JASIF,2023-10-11,2023-10-11,6.65,6.80,17166120,6.80,6.65,2.26,B,6.80,0.00,2007626,2023-10-12
3,JMT,2023-10-06,2023-10-11,46.25,40.75,269784432,50.00,40.75,-11.89,B,38.00,-2.75,34362774,2023-10-12
4,NER,2023-10-10,2023-10-11,4.46,4.56,20767712,4.56,4.46,2.24,B,4.66,0.10,9437514,2023-10-12
5,ORI,2023-10-10,2023-10-11,9.75,10.20,13089996,10.20,9.75,4.62,B,10.30,0.10,2445753,2023-10-12
6,PTG,2023-10-11,2023-10-11,8.80,9.00,36446964,9.20,8.80,2.27,B,8.90,-0.10,3498669,2023-10-12
7,PTTGC,2023-10-10,2023-10-11,33.00,34.25,59101876,34.25,31.75,3.79,B,33.75,-0.50,9765663,2023-10-12
8,SENA,2023-10-11,2023-10-11,2.64,2.66,25069508,2.80,2.64,0.76,B,2.64,-0.02,512550,2023-10-12
9,SINGER,2023-10-11,2023-10-11,11.30,11.60,158941944,13.40,11.30,2.65,B,11.00,-0.60,12286055,2023-10-12


In [16]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Call ruby ruby\daily-out-new.rb

In [17]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [18]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [19]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-10-12 2023-10-12  33.75  34.25   1.48%   4,823,876  34.50  32.25 I Insert
AIMIRT   2023-10-12 2023-10-12  10.50  10.40  -0.95%   5,916,784  10.60  10.40 I Insert
ASK      2023-10-12 2023-10-12  21.30  21.20  -0.47%   2,903,164  22.00  20.30 B Insert
ASP      2023-10-11 2023-10-12   2.80   2.82   0.71%   1,440,650   2.84   2.78 I Update
AWC      2023-10-12 2023-10-12   3.70   3.68  -0.54% 639,602,652   3.76   3.58 I Insert
BBL      2023-10-11 2023-10-12 166.50 169.50    1.8%   9,192,846 169.50 166.50 O Update
BCH      2023-10-12 2023-10-12  20.30  20.20  -0.49%  25,342,848  20.40  19.90 I Insert
BEM      2023-10-06 2023-10-12   8.00   8.20    2.5%  84,444,008   8.20   7.95 O Update
BH       2023-10-11 2023-10-12 254.00 264.00   3.94%  31,253,044 269.00 254.00 O Update
CK       2023-10-11 2023-10-12  

In [20]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb

In [22]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-10-12' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [23]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,ASK,2023-10-12,2023-10-12,21.30,21.20,"2,903,164",22.00,20.30,B,SET,-0.10,-0.47%
1,DIF,2023-10-12,2023-10-12,9.10,9.05,"46,019,438",9.10,9.00,B,SET,-0.05,-0.55%
2,JASIF,2023-10-11,2023-10-12,6.65,6.80,"34,332,240",6.80,6.65,B,SET,0.15,2.26%
3,JMT,2023-10-06,2023-10-12,46.25,38.00,"539,568,864",50.00,38.00,B,SET50,-8.25,-17.84%
4,NER,2023-10-10,2023-10-12,4.46,4.66,"41,535,424",4.66,4.46,B,SET,0.20,4.48%
5,ORI,2023-10-10,2023-10-12,9.75,10.30,"26,179,992",10.30,9.75,B,SET100,0.55,5.64%
6,PTG,2023-10-12,2023-10-12,9.00,8.90,"36,446,964",9.20,8.80,B,SET100,-0.10,-1.11%
7,PTTGC,2023-10-12,2023-10-12,34.25,33.75,"59,101,876",34.25,31.75,B,SET50,-0.50,-1.46%
8,SENA,2023-10-12,2023-10-12,2.66,2.64,"25,069,508",2.80,2.64,B,SET,-0.02,-0.75%
9,SINGER,2023-10-12,2023-10-12,11.60,11.00,"158,941,944",13.40,11.00,B,SET100,-0.60,-5.17%


In [24]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [25]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
3,JMT,2023-10-06,2023-10-12,46.25,38.00,"539,568,864",50.00,38.00,B,SET50,-8.25,-17.84%
9,SINGER,2023-10-12,2023-10-12,11.60,11.00,"158,941,944",13.40,11.00,B,SET100,-0.60,-5.17%


In [26]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
21,IVL,2023-10-10,2023-10-12,24.40,25.75,"109,476,520",25.75,24.40,I,SET50,1.35,5.53%
27,STA,2023-10-10,2023-10-12,14.30,15.10,"54,041,788",15.10,14.30,I,SET100,0.80,5.59%
5,ORI,2023-10-10,2023-10-12,9.75,10.30,"26,179,992",10.30,9.75,B,SET100,0.55,5.64%
44,SC,2023-10-09,2023-10-12,3.82,4.10,"137,669,840",4.10,3.80,O,SET,0.28,7.33%
